In [4]:
batch_size = 32

In [8]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

train_datagen = ImageDataGenerator(
    rotation_range=360,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator()

train_gen = train_datagen.flow_from_directory(
    'images/train',
    target_size=(224, 224),
    batch_size=batch_size
)
val_den = test_datagen.flow_from_directory(
    'images/val',
    target_size=(224, 224),
    batch_size=batch_size
)
test_gen = test_datagen.flow_from_directory(
    'images/test',
    target_size=(224, 224),
    batch_size=batch_size
)

Found 662913 images belonging to 92 classes.
Found 221021 images belonging to 92 classes.


In [14]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.layers import Flatten, Dense, Input
from keras.models import Model
import numpy as np

num_classes = 92
inp = Input(shape=(224, 224, 3))
base_model = ResNet50(input_tensor=inp, weights='imagenet', include_top=False, classes=num_classes)
last_layer = base_model.output

# Add flatten and dense layers
last_layer = Flatten(name='flatten_2')(last_layer)
out = Dense(num_classes, activation='softmax', name='output_layer')(last_layer)

model = Model(inputs=inp, outputs=out)
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [19]:
model.fit_generator(train_gen, steps_per_epoch=250, epochs=4, verbose=1)

Epoch 1/4
111/250 [============>.................] - ETA: 10:30 - loss: 1.4702 - acc: 0.6402

KeyboardInterrupt: 

In [18]:
model.evaluate_generator(test_gen, 2000)

KeyboardInterrupt: 